In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
from scipy.linalg import svd
from scipy.spatial.transform import Rotation
from scipy import signal
import io
from datetime import datetime
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# 全局变量
calibration_results = {}  # 存储校准结果
single_imu_motion_data = {}  # 存储每个IMU的运动数据

class SingleIMUDataLoader:
    def __init__(self):
        self.static_data = None
        self.rotation_data = None
        self.sample_rate = 30.0
        self.current_device_name = None
        
        # Filter settings
        self.filter_enabled = True
        self.filter_cutoff = 10.0
        self.filter_order = 4
        
        # 新数据格式的列名
        # 静态校准用包含重力的加速度
        self.static_accel_columns = ['Ax(m/s²)', 'Ay(m/s²)', 'Az(m/s²)']  # 包含重力
        # 运动分析用姿态相关的加速度
        self.motion_accel_columns = ['aX(m/s²)', 'aY(m/s²)', 'aZ(m/s²)']  # 姿态相关
        self.gyro_columns = ['Gx(°/s)', 'Gy(°/s)', 'Gz(°/s)']
        
        # 不需要转换因子，因为加速度已经是m/s²
        self.accel_conversion = 1.0  # m/s² to m/s²
        
    def apply_lowpass_filter(self, data, cutoff_freq, order):
        """Apply Butterworth lowpass filter to data"""
        if not self.filter_enabled or cutoff_freq >= self.sample_rate / 2:
            return data
            
        nyquist = self.sample_rate / 2
        normalized_cutoff = cutoff_freq / nyquist
        b, a = signal.butter(order, normalized_cutoff, btype='low', analog=False)
        
        filtered_data = np.zeros_like(data)
        for i in range(data.shape[1]):
            filtered_data[:, i] = signal.filtfilt(b, a, data[:, i])
        
        return filtered_data
    
    def check_and_clean_data(self, df, data_type):
        """Check for NaN/Inf values and clean the data"""
        nan_counts = {}
        inf_counts = {}
        
        # 根据数据类型选择正确的列
        if data_type == 'static':
            accel_cols = self.static_accel_columns
        else:
            accel_cols = self.motion_accel_columns
            
        for col in accel_cols + self.gyro_columns:
            if col in df.columns:
                nan_counts[col] = df[col].isna().sum()
                inf_counts[col] = np.isinf(df[col]).sum()
        
        total_nans = sum(nan_counts.values())
        total_infs = sum(inf_counts.values())
        
        if total_nans > 0 or total_infs > 0:
            print(f"\n⚠️ Data quality issues detected in {data_type} data:")
            print(f"Total NaN values: {total_nans}")
            print(f"Total Inf values: {total_infs}")
            
            # Replace inf with NaN
            for col in accel_cols + self.gyro_columns:
                if col in df.columns:
                    df.loc[np.isinf(df[col]), col] = np.nan
            
            # Interpolate
            for col in accel_cols + self.gyro_columns:
                if col in df.columns:
                    df[col] = df[col].interpolate(method='linear', limit_direction='both')
                    df[col] = df[col].fillna(method='ffill').fillna(method='bfill')
                    
                    if df[col].isna().any():
                        print(f"  Warning: Column {col} has all NaN values, filling with 0")
                        df[col] = df[col].fillna(0)
            
            print(f"✅ Data cleaning successful!")
        
        return df
    
    def load_data(self, file_content, data_type):
        """Load IMU data file with new format"""
        try:
            # Handle different file content types
            if isinstance(file_content, bytes):
                file_content_str = file_content.decode('utf-8')
                df = pd.read_csv(io.StringIO(file_content_str), skipinitialspace=True)
            elif isinstance(file_content, str):
                if os.path.exists(file_content):
                    df = pd.read_csv(file_content, skipinitialspace=True)
                else:
                    df = pd.read_csv(io.StringIO(file_content), skipinitialspace=True)
            else:
                df = pd.read_csv(io.BytesIO(file_content), skipinitialspace=True)
            
            # Strip whitespace from column names
            df.columns = df.columns.str.strip()
            
            # Print available columns for debugging
            print(f"Available columns: {list(df.columns)}")
            
            # Check for required columns based on data type
            missing_cols = []
            if data_type == 'static':
                # 静态校准需要包含重力的加速度
                print(f"Loading static data, expecting columns: {self.static_accel_columns}")
                for col in self.static_accel_columns + self.gyro_columns:
                    if col not in df.columns:
                        missing_cols.append(col)
            else:
                # 旋转校准和运动分析需要姿态相关的加速度
                print(f"Loading {data_type} data, expecting columns: {self.motion_accel_columns}")
                for col in self.motion_accel_columns + self.gyro_columns:
                    if col not in df.columns:
                        missing_cols.append(col)
            
            if missing_cols:
                raise ValueError(f"Missing required columns for {data_type}: {missing_cols}")
            
            # Clean the data
            df = self.check_and_clean_data(df, data_type)
            
            # Store the dataframe
            if data_type == 'static':
                self.static_data = df
            elif data_type == 'rotation':
                self.rotation_data = df
            
            return df
            
        except Exception as e:
            print(f"Error loading data: {e}")
            return None
    
    def get_processed_data(self, data_type, apply_filter=True):
        """Get processed data with optional filtering"""
        if data_type == 'static' and self.static_data is not None:
            df = self.static_data.copy()
            accel_cols = self.static_accel_columns
        elif data_type == 'rotation' and self.rotation_data is not None:
            df = self.rotation_data.copy()
            accel_cols = self.motion_accel_columns
        else:
            return None
        
        # Apply lowpass filter if requested
        if apply_filter and self.filter_enabled:
            # Filter acceleration data
            accel_data = df[accel_cols].values
            filtered_accel = self.apply_lowpass_filter(accel_data, self.filter_cutoff, self.filter_order)
            df[accel_cols] = filtered_accel
            
            # Filter gyroscope data
            gyro_data = df[self.gyro_columns].values
            filtered_gyro = self.apply_lowpass_filter(gyro_data, self.filter_cutoff, self.filter_order)
            df[self.gyro_columns] = filtered_gyro
        
        return df
    
    def get_static_accel(self):
        """Get static acceleration data (包含重力的加速度，已经是m/s²)"""
        df = self.get_processed_data('static', apply_filter=True)
        if df is None or len(df) == 0:
            return None
        
        # Get acceleration data (使用包含重力的Ax, Ay, Az)
        accel_data = df[self.static_accel_columns].values * self.accel_conversion
        
        # Final check for NaN/Inf
        if np.any(np.isnan(accel_data)) or np.any(np.isinf(accel_data)):
            print("Warning: NaN or Inf values detected in static acceleration data")
            mask = ~(np.any(np.isnan(accel_data), axis=1) | np.any(np.isinf(accel_data), axis=1))
            accel_data = accel_data[mask]
            print(f"Removed {np.sum(~mask)} rows with bad data")
        
        return accel_data
    
    def get_rotation_gyro(self):
        """Get rotation gyroscope data (converted to rad/s)"""
        df = self.get_processed_data('rotation', apply_filter=True)
        if df is None or len(df) == 0:
            return None
        
        # Convert to rad/s
        gyro_data = df[self.gyro_columns].values * (np.pi / 180.0)
        
        # Final check for NaN/Inf
        if np.any(np.isnan(gyro_data)) or np.any(np.isinf(gyro_data)):
            print("Warning: NaN or Inf values detected in rotation gyroscope data")
            mask = ~(np.any(np.isnan(gyro_data), axis=1) | np.any(np.isinf(gyro_data), axis=1))
            gyro_data = gyro_data[mask]
            print(f"Removed {np.sum(~mask)} rows with bad data")
        
        return gyro_data
    
    def preview_data(self, data_type):
        """Preview acceleration and gyroscope data with filter comparison"""
        df_filtered = self.get_processed_data(data_type, apply_filter=True)
        df_unfiltered = self.get_processed_data(data_type, apply_filter=False)
        
        if df_filtered is None or len(df_filtered) == 0:
            print(f"No data available for {data_type}")
            return
        
        # 根据数据类型选择正确的加速度列
        if data_type == 'static':
            accel_cols = self.static_accel_columns
        else:
            accel_cols = self.motion_accel_columns
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))
        fig.suptitle(f'{data_type.capitalize()} Data Preview - {self.current_device_name}', fontsize=14, weight='bold')
        
        # Use timestamp if available, otherwise create time array
        if 'timestamp' in df_filtered.columns:
            time = (df_filtered['timestamp'].values - df_filtered['timestamp'].values[0]) / 1000.0  # Convert to seconds
        else:
            time = np.arange(len(df_filtered)) / self.sample_rate
        
        colors = ['red', 'green', 'blue']
        labels = ['X', 'Y', 'Z']
        
        # Unfiltered acceleration
        for i, (col, color, label) in enumerate(zip(accel_cols, colors, labels)):
            axes[0, 0].plot(time, df_unfiltered[col].values, color=color, 
                          label=f'Accel {label}', linewidth=1.5, alpha=0.7)
        
        axes[0, 0].set_xlabel('Time (s)')
        axes[0, 0].set_ylabel('Acceleration (m/s²)')
        axes[0, 0].set_title('Acceleration Data - Unfiltered')
        axes[0, 0].legend(loc='upper right')
        axes[0, 0].grid(True, alpha=0.3)
        
        # Filtered acceleration
        for i, (col, color, label) in enumerate(zip(accel_cols, colors, labels)):
            axes[0, 1].plot(time, df_filtered[col].values, color=color, 
                          label=f'Accel {label}', linewidth=1.5)
        
        axes[0, 1].set_xlabel('Time (s)')
        axes[0, 1].set_ylabel('Acceleration (m/s²)')
        axes[0, 1].set_title(f'Acceleration Data - Filtered (fc={self.filter_cutoff}Hz, order={self.filter_order})')
        axes[0, 1].legend(loc='upper right')
        axes[0, 1].grid(True, alpha=0.3)
        
        # Unfiltered gyroscope
        for i, (col, color, label) in enumerate(zip(self.gyro_columns, colors, labels)):
            axes[1, 0].plot(time, df_unfiltered[col].values, color=color, 
                          label=f'Gyro {label}', linewidth=1.5, alpha=0.7)
        
        axes[1, 0].set_xlabel('Time (s)')
        axes[1, 0].set_ylabel('Angular Velocity (°/s)')
        axes[1, 0].set_title('Gyroscope Data - Unfiltered')
        axes[1, 0].legend(loc='upper right')
        axes[1, 0].grid(True, alpha=0.3)
        
        # Filtered gyroscope
        for i, (col, color, label) in enumerate(zip(self.gyro_columns, colors, labels)):
            axes[1, 1].plot(time, df_filtered[col].values, color=color, 
                          label=f'Gyro {label}', linewidth=1.5)
        
        axes[1, 1].set_xlabel('Time (s)')
        axes[1, 1].set_ylabel('Angular Velocity (°/s)')
        axes[1, 1].set_title(f'Gyroscope Data - Filtered (fc={self.filter_cutoff}Hz, order={self.filter_order})')
        axes[1, 1].legend(loc='upper right')
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n=== Data Summary for {self.current_device_name} ===")
        print(f"Data type: {data_type}")
        print(f"Duration: {time[-1]:.2f} seconds")
        print(f"Samples: {len(df_filtered)}")
        print(f"Sample rate: {self.sample_rate} Hz")
        if self.filter_enabled:
            print(f"Filter: Butterworth low-pass, cutoff={self.filter_cutoff}Hz, order={self.filter_order}")

def calibrate_imu_to_board(static_accel_data, rotation_gyro_data):
    """
    Calculate rotation matrix between IMU and snowboard
    
    Args:
        static_accel_data: Acceleration data during static placement (N×3 array, m/s²)
                          包含重力的加速度，当IMU z轴朝上时读数为+g（反作用力）
        rotation_gyro_data: Gyroscope data during rotation (M×3 array, rad/s)
    
    Returns:
        R_board_to_imu: 3×3 rotation matrix (transforms vectors from board to IMU frame)
        installation_angles: Installation angles (roll, pitch, yaw) in degrees
    """
    # Validate input data
    if np.any(np.isnan(static_accel_data)) or np.any(np.isinf(static_accel_data)):
        raise ValueError("Static acceleration data contains NaN or Inf values")
    
    if np.any(np.isnan(rotation_gyro_data)) or np.any(np.isinf(rotation_gyro_data)):
        raise ValueError("Rotation gyroscope data contains NaN or Inf values")
    
    # Step 1: Static Calibration
    # 平均加速度向量指向"上"的方向（重力的反作用力）
    avg_accel = np.mean(static_accel_data, axis=0)
    avg_accel_magnitude = np.linalg.norm(avg_accel)
    zb_imu = avg_accel / avg_accel_magnitude
    
    print(f"Average acceleration: {avg_accel}")
    print(f"Average acceleration magnitude: {avg_accel_magnitude:.3f} m/s² (expected ~9.8)")
    print(f"Board normal (zb) in IMU frame: {zb_imu}")
    
    # Step 2: Rotation Calibration
    U, s, Vt = svd(rotation_gyro_data, full_matrices=False)
    xb_imu = Vt[0]
    xb_imu = xb_imu / np.linalg.norm(xb_imu)
    
    print(f"Rotation axis (xb) in IMU frame: {xb_imu}")
    print(f"Singular values: {s}")
    
    # Orthogonalization
    dot_product = np.dot(zb_imu, xb_imu)
    if abs(dot_product) > 0.1:
        print(f"Warning: zb and xb not orthogonal (dot={dot_product:.3f}), performing orthogonalization")
        xb_imu = xb_imu - dot_product * zb_imu
        xb_imu = xb_imu / np.linalg.norm(xb_imu)
    
    # Calculate yb axis
    yb_imu = np.cross(zb_imu, xb_imu)
    yb_norm = np.linalg.norm(yb_imu)
    if yb_norm < 1e-6:
        raise ValueError("Cross product invalid, zb and xb may be parallel")
    yb_imu = yb_imu / yb_norm
    
    # Construct Rotation Matrix
    R_board_to_imu = np.column_stack((xb_imu, yb_imu, zb_imu))
    
    # Validate rotation matrix
    det = np.linalg.det(R_board_to_imu)
    if abs(det - 1) > 0.1:
        print(f"Warning: Rotation matrix determinant={det:.3f} (expected 1)")
    
    # Calculate installation angles
    rotation = Rotation.from_matrix(R_board_to_imu)
    installation_angles = rotation.as_euler('xyz', degrees=True)
    
    return R_board_to_imu, installation_angles

class SingleIMUMotionAnalyzer:
    def __init__(self, calibration_results):
        self.calibration_results = calibration_results
        self.sample_rate = 30.0
        
        # Filter settings
        self.filter_enabled = True
        self.filter_cutoff = 10.0
        self.filter_order = 4
        
        # 列名设置
        self.accel_columns = ['aX(m/s²)', 'aY(m/s²)', 'aZ(m/s²)']  # 姿态相关的加速度
        self.gyro_columns = ['Gx(°/s)', 'Gy(°/s)', 'Gz(°/s)']
        
        # Angle inversion settings - format: {device_name: {'roll': invert, 'pitch': invert, 'yaw': invert}}
        self.angle_inversions = {}
        
    def apply_lowpass_filter(self, data, cutoff_freq, order):
        """Apply Butterworth lowpass filter to data"""
        if not self.filter_enabled or cutoff_freq >= self.sample_rate / 2:
            return data
            
        nyquist = self.sample_rate / 2
        normalized_cutoff = cutoff_freq / nyquist
        b, a = signal.butter(order, normalized_cutoff, btype='low', analog=False)
        
        filtered_data = np.zeros_like(data)
        for i in range(data.shape[1]):
            filtered_data[:, i] = signal.filtfilt(b, a, data[:, i])
        
        return filtered_data
    
    def process_single_imu(self, file_content, device_name):
        """Process single IMU motion data"""
        try:
            # Load data
            if isinstance(file_content, bytes):
                file_content_str = file_content.decode('utf-8')
                df = pd.read_csv(io.StringIO(file_content_str), skipinitialspace=True)
            else:
                df = pd.read_csv(io.BytesIO(file_content), skipinitialspace=True)
            
            # Strip whitespace from column names
            df.columns = df.columns.str.strip()
            
            # Check for required columns
            missing_cols = []
            for col in self.accel_columns + self.gyro_columns:
                if col not in df.columns:
                    missing_cols.append(col)
            
            if missing_cols:
                raise ValueError(f"Missing required columns: {missing_cols}")
            
            # Check if device has calibration
            if device_name not in self.calibration_results:
                raise ValueError(f"No calibration data for {device_name}")
            
            # Get calibration parameters
            calib = self.calibration_results[device_name]
            R_imu_to_board = calib['R_imu_to_board']
            
            # Get time array - prioritize 'time' column for better alignment
            absolute_time = None
            if 'time' in df.columns:
                try:
                    # Parse time strings (HH:MM:SS.mmm format)
                    time_strings = df['time'].astype(str)
                    # Convert to seconds since midnight
                    absolute_time = []
                    for time_str in time_strings:
                        parts = time_str.split(':')
                        if len(parts) == 3:
                            hours = float(parts[0])
                            minutes = float(parts[1])
                            seconds = float(parts[2])
                            total_seconds = hours * 3600 + minutes * 60 + seconds
                            absolute_time.append(total_seconds)
                    absolute_time = np.array(absolute_time)
                    # Make relative to first timestamp
                    time_array = absolute_time - absolute_time[0]
                    print(f"   Using 'time' column for precise alignment (first time: {time_strings.iloc[0]})")
                except Exception as e:
                    print(f"   Warning: Failed to parse 'time' column: {e}")
                    absolute_time = None
            
            # Fallback to timestamp column
            if absolute_time is None:
                if 'timestamp' in df.columns:
                    time_array = (df['timestamp'].values - df['timestamp'].values[0]) / 1000.0
                    # Store absolute timestamp for alignment
                    absolute_time = df['timestamp'].values / 1000.0
                    print(f"   Using 'timestamp' column")
                else:
                    num_samples = len(df)
                    time_array = np.arange(0, num_samples / self.sample_rate, 1 / self.sample_rate)[:num_samples]
                    print(f"   No time column found, using sample rate")
            
            # Get IMU data
            accel_imu_raw = df[self.accel_columns].values  # Already in m/s²
            gyro_imu_raw = df[self.gyro_columns].values * (np.pi / 180.0)  # Convert to rad/s
            
            # Apply filter if enabled
            if self.filter_enabled:
                accel_imu = self.apply_lowpass_filter(accel_imu_raw, self.filter_cutoff, self.filter_order)
                gyro_imu = self.apply_lowpass_filter(gyro_imu_raw, self.filter_cutoff, self.filter_order)
            else:
                accel_imu = accel_imu_raw
                gyro_imu = gyro_imu_raw
            
            # Convert to board frame
            accel_board = np.array([R_imu_to_board @ a for a in accel_imu])
            gyro_board = np.array([R_imu_to_board @ g for g in gyro_imu])
            
            # Store results
            single_imu_motion_data[device_name] = {
                'time': time_array,
                'absolute_time': absolute_time,  # Store for alignment
                'accel_board': accel_board,
                'gyro_board': gyro_board,
                'accel_imu_raw': accel_imu_raw,
                'gyro_imu_raw': gyro_imu_raw,
                'device_name': device_name,
                'upload_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                'filter_settings': {
                    'enabled': self.filter_enabled,
                    'cutoff': self.filter_cutoff,
                    'order': self.filter_order
                }
            }
            
            print(f"✅ Successfully processed {device_name}: {len(time_array)} samples")
            if self.filter_enabled:
                print(f"   Applied filter: Butterworth low-pass, cutoff={self.filter_cutoff}Hz, order={self.filter_order}")
            
            return True
            
        except Exception as e:
            print(f"❌ Error processing {device_name}: {e}")
            import traceback
            traceback.print_exc()
            return False
    
    def quaternion_multiply(self, q1, q2):
        """Quaternion multiplication"""
        w1, x1, y1, z1 = q1
        w2, x2, y2, z2 = q2
        
        return np.array([
            w1*w2 - x1*x2 - y1*y2 - z1*z2,
            w1*x2 + x1*w2 + y1*z2 - z1*y2,
            w1*y2 - x1*z2 + y1*w2 + z1*x2,
            w1*z2 + x1*y2 - y1*x2 + z1*w2
        ])
    
    def align_timestamps(self, data_dict, selected_devices):
        """Align timestamps across multiple IMUs using absolute time references"""
        # Check if we have absolute time information
        has_absolute_time = all(['absolute_time' in data_dict[device] for device in selected_devices])
        
        if has_absolute_time:
            print("   Using absolute time for precise alignment")
            # Find the common absolute time range
            min_abs_time = max([data_dict[device]['absolute_time'][0] for device in selected_devices])
            max_abs_time = min([data_dict[device]['absolute_time'][-1] for device in selected_devices])
            
            # Create common absolute time array
            common_dt = 1.0 / self.sample_rate
            common_abs_time = np.arange(min_abs_time, max_abs_time, common_dt)
            
            # Create relative time array
            common_time = common_abs_time - common_abs_time[0]
            
            # Interpolate data for each device using absolute time
            aligned_data = {}
            for device in selected_devices:
                data = data_dict[device]
                original_abs_time = data['absolute_time']
                
                # Interpolate gyro data
                gyro_aligned = np.zeros((len(common_abs_time), 3))
                for i in range(3):
                    gyro_aligned[:, i] = np.interp(common_abs_time, original_abs_time, data['gyro_board'][:, i])
                
                # Interpolate acceleration data
                accel_aligned = np.zeros((len(common_abs_time), 3))
                for i in range(3):
                    accel_aligned[:, i] = np.interp(common_abs_time, original_abs_time, data['accel_board'][:, i])
                
                aligned_data[device] = {
                    'time': common_time,
                    'gyro_board': gyro_aligned,
                    'accel_board': accel_aligned
                }
            
            # Report alignment details
            print(f"   Aligned to common time range: {common_time[0]:.3f}s to {common_time[-1]:.3f}s")
            print(f"   Absolute time range: {min_abs_time:.3f}s to {max_abs_time:.3f}s")
            
        else:
            print("   Using relative time alignment (no absolute time available)")
            # Fallback to original relative time alignment
            min_start_time = max([data_dict[device]['time'][0] for device in selected_devices])
            max_end_time = min([data_dict[device]['time'][-1] for device in selected_devices])
            
            common_dt = 1.0 / self.sample_rate
            common_time = np.arange(min_start_time, max_end_time, common_dt)
            
            aligned_data = {}
            for device in selected_devices:
                data = data_dict[device]
                original_time = data['time']
                
                # Interpolate gyro data
                gyro_aligned = np.zeros((len(common_time), 3))
                for i in range(3):
                    gyro_aligned[:, i] = np.interp(common_time, original_time, data['gyro_board'][:, i])
                
                # Interpolate acceleration data
                accel_aligned = np.zeros((len(common_time), 3))
                for i in range(3):
                    accel_aligned[:, i] = np.interp(common_time, original_time, data['accel_board'][:, i])
                
                aligned_data[device] = {
                    'time': common_time,
                    'gyro_board': gyro_aligned,
                    'accel_board': accel_aligned
                }
        
        return aligned_data, common_time
    
    
    def plot_multi_imu_comparison(self, selected_devices):
        """Plot comparison of multiple IMUs (1-8 devices) with aligned timestamps"""
        if len(selected_devices) == 0:
            print("No devices selected for comparison")
            return
        
        # Define colors for up to 8 IMUs
        colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink', 'gray']
        
        # Align timestamps
        print("Aligning timestamps across devices...")
        aligned_data, common_time = self.align_timestamps(single_imu_motion_data, selected_devices)
        
        # Calculate Euler angles for each IMU using aligned data
        euler_angles_dict = {}
        
        for device in selected_devices:
            if device not in aligned_data:
                continue
                
            gyro_board = aligned_data[device]['gyro_board']
            
            # Use quaternion integration
            dt = 1.0 / self.sample_rate
            euler_angles = []
            
            # Initial attitude
            q = np.array([1.0, 0.0, 0.0, 0.0])
            euler_angles.append([0.0, 0.0, 0.0])
            
            # Integrate angular velocity
            for i in range(1, len(common_time)):
                omega = gyro_board[i]
                omega_norm = np.linalg.norm(omega)
                
                if omega_norm > 1e-10:
                    axis = omega / omega_norm
                    angle = omega_norm * dt
                    dq = np.array([
                        np.cos(angle/2),
                        axis[0] * np.sin(angle/2),
                        axis[1] * np.sin(angle/2),
                        axis[2] * np.sin(angle/2)
                    ])
                else:
                    dq = np.array([1.0, 0.0, 0.0, 0.0])
                
                # Update quaternion
                q = self.quaternion_multiply(q, dq)
                q = q / np.linalg.norm(q)
                
                # Convert to Euler angles
                r = Rotation.from_quat([q[1], q[2], q[3], q[0]])
                euler = r.as_euler('xyz', degrees=True)
                euler_angles.append(euler)
            
            euler_angles_dict[device] = np.array(euler_angles)
        
        # Create plots with 50% larger size (27x15 instead of 18x10)
        n_devices = len(selected_devices)
        fig, axes = plt.subplots(2, 3, figsize=(27, 15))
        fig.suptitle(f'Multi-IMU Comparison ({n_devices} devices) - Board Frame (Time Aligned)', fontsize=20)
        
        # Plot each device
        for idx, device in enumerate(selected_devices):
            if device not in euler_angles_dict:
                continue
            
            color = colors[idx % len(colors)]
            device_label = device if len(device) <= 20 else device[:17] + "..."
            
            # Get inversion settings for this device
            device_inversions = self.angle_inversions.get(device, {})
            roll_factor = -1 if device_inversions.get('roll', False) else 1
            pitch_factor = -1 if device_inversions.get('pitch', False) else 1
            yaw_factor = -1 if device_inversions.get('yaw', False) else 1
            gyro_x_factor = -1 if device_inversions.get('gyro_x', False) else 1
            gyro_y_factor = -1 if device_inversions.get('gyro_y', False) else 1
            gyro_z_factor = -1 if device_inversions.get('gyro_z', False) else 1
            
            # Add inversion indicator to label
            inversions = []
            if roll_factor == -1: inversions.append('R')
            if pitch_factor == -1: inversions.append('P')
            if yaw_factor == -1: inversions.append('Y')
            if inversions:
                device_label += f" (-{','.join(inversions)})"
            
            # Row 1: Euler angles
            # Roll
            axes[0, 0].plot(common_time, 
                           euler_angles_dict[device][:, 0] * roll_factor, 
                           color=color, label=device_label, linewidth=2.5, alpha=0.8)
            
            # Pitch
            axes[0, 1].plot(common_time, 
                           euler_angles_dict[device][:, 1] * pitch_factor, 
                           color=color, label=device_label, linewidth=2.5, alpha=0.8)
            
            # Yaw
            axes[0, 2].plot(common_time, 
                           euler_angles_dict[device][:, 2] * yaw_factor, 
                           color=color, label=device_label, linewidth=2.5, alpha=0.8)
            
            # Row 2: Angular velocities
            # X-axis
            axes[1, 0].plot(common_time, 
                           aligned_data[device]['gyro_board'][:, 0] * 180/np.pi * gyro_x_factor, 
                           color=color, label=device_label, linewidth=2.5, alpha=0.8)
            
            # Y-axis
            axes[1, 1].plot(common_time, 
                           aligned_data[device]['gyro_board'][:, 1] * 180/np.pi * gyro_y_factor, 
                           color=color, label=device_label, linewidth=2.5, alpha=0.8)
            
            # Z-axis
            axes[1, 2].plot(common_time, 
                           aligned_data[device]['gyro_board'][:, 2] * 180/np.pi * gyro_z_factor, 
                           color=color, label=device_label, linewidth=2.5, alpha=0.8)
        
        # Configure axes with larger font sizes
        axes[0, 0].set_xlabel('Time (s)', fontsize=14)
        axes[0, 0].set_ylabel('Roll (deg)', fontsize=14)
        axes[0, 0].set_title('Roll Angle', fontsize=16)
        axes[0, 0].legend(fontsize=10, loc='best')
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].tick_params(labelsize=12)
        
        axes[0, 1].set_xlabel('Time (s)', fontsize=14)
        axes[0, 1].set_ylabel('Pitch (deg)', fontsize=14)
        axes[0, 1].set_title('Pitch Angle', fontsize=16)
        axes[0, 1].legend(fontsize=10, loc='best')
        axes[0, 1].grid(True, alpha=0.3)
        axes[0, 1].tick_params(labelsize=12)
        
        axes[0, 2].set_xlabel('Time (s)', fontsize=14)
        axes[0, 2].set_ylabel('Yaw (deg)', fontsize=14)
        axes[0, 2].set_title('Yaw Angle', fontsize=16)
        axes[0, 2].legend(fontsize=10, loc='best')
        axes[0, 2].grid(True, alpha=0.3)
        axes[0, 2].tick_params(labelsize=12)
        
        axes[1, 0].set_xlabel('Time (s)', fontsize=14)
        axes[1, 0].set_ylabel('omega_x (deg/s)', fontsize=14)
        axes[1, 0].set_title('X-axis Angular Velocity', fontsize=16)
        axes[1, 0].legend(fontsize=10, loc='best')
        axes[1, 0].grid(True, alpha=0.3)
        axes[1, 0].tick_params(labelsize=12)
        
        axes[1, 1].set_xlabel('Time (s)', fontsize=14)
        axes[1, 1].set_ylabel('omega_y (deg/s)', fontsize=14)
        axes[1, 1].set_title('Y-axis Angular Velocity', fontsize=16)
        axes[1, 1].legend(fontsize=10, loc='best')
        axes[1, 1].grid(True, alpha=0.3)
        axes[1, 1].tick_params(labelsize=12)
        
        axes[1, 2].set_xlabel('Time (s)', fontsize=14)
        axes[1, 2].set_ylabel('omega_z (deg/s)', fontsize=14)
        axes[1, 2].set_title('Z-axis Angular Velocity', fontsize=16)
        axes[1, 2].legend(fontsize=10, loc='best')
        axes[1, 2].grid(True, alpha=0.3)
        axes[1, 2].tick_params(labelsize=12)
        
        # Add filter info with larger font
        if self.filter_enabled:
            filter_text = f"Filter: Butterworth LP, fc={self.filter_cutoff}Hz, order={self.filter_order}"
            fig.text(0.5, 0.94, filter_text, ha='center', fontsize=12, 
                    bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
        
        # Add timestamp alignment info
        has_absolute_time = all(['absolute_time' in single_imu_motion_data[device] for device in selected_devices])
        if has_absolute_time:
            alignment_text = f"Time aligned using absolute timestamps: {common_time[0]:.2f}s to {common_time[-1]:.2f}s"
        else:
            alignment_text = f"Time aligned using relative timestamps: {common_time[0]:.2f}s to {common_time[-1]:.2f}s"
        fig.text(0.5, 0.92, alignment_text, ha='center', fontsize=12, 
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
        
        plt.tight_layout()
        plt.show()
        
        # Statistical analysis
        print("\n=== Multi-IMU Statistical Analysis ===")
        print(f"Devices compared: {n_devices}")
        
        has_absolute_time = all(['absolute_time' in single_imu_motion_data[device] for device in selected_devices])
        if has_absolute_time:
            print("Time alignment method: Absolute time (using 'time' column)")
        else:
            print("Time alignment method: Relative time")
            
        print(f"Time-aligned duration: {common_time[-1] - common_time[0]:.2f} seconds")
        print(f"Common samples: {len(common_time)}")
        
        for device in selected_devices:
            if device in single_imu_motion_data:
                original_duration = single_imu_motion_data[device]['time'][-1] - single_imu_motion_data[device]['time'][0]
                original_samples = len(single_imu_motion_data[device]['time'])
                print(f"  {device}: {original_samples} original samples, {original_duration:.2f} seconds")
        
        if n_devices > 1:
            print("\n=== Pairwise Comparisons (Time-Aligned) ===")
            # Compare each pair of devices
            for i in range(n_devices):
                for j in range(i+1, n_devices):
                    device1 = selected_devices[i]
                    device2 = selected_devices[j]
                    
                    if device1 not in euler_angles_dict or device2 not in euler_angles_dict:
                        continue
                    
                    # Apply inversions for comparison
                    device1_inversions = self.angle_inversions.get(device1, {})
                    device2_inversions = self.angle_inversions.get(device2, {})
                    
                    # Get euler angles with inversions applied
                    euler1 = euler_angles_dict[device1].copy()
                    euler2 = euler_angles_dict[device2].copy()
                    
                    if device1_inversions.get('roll', False): euler1[:, 0] *= -1
                    if device1_inversions.get('pitch', False): euler1[:, 1] *= -1
                    if device1_inversions.get('yaw', False): euler1[:, 2] *= -1
                    
                    if device2_inversions.get('roll', False): euler2[:, 0] *= -1
                    if device2_inversions.get('pitch', False): euler2[:, 1] *= -1
                    if device2_inversions.get('yaw', False): euler2[:, 2] *= -1
                    
                    # Euler angle differences (already same length due to alignment)
                    euler_diff = euler1 - euler2
                    
                    print(f"\n{device1} vs {device2}:")
                    print("  Euler Angle RMS Differences:")
                    print(f"    Roll:  {np.sqrt(np.mean(euler_diff[:, 0]**2)):.2f} deg")
                    print(f"    Pitch: {np.sqrt(np.mean(euler_diff[:, 1]**2)):.2f} deg")
                    print(f"    Yaw:   {np.sqrt(np.mean(euler_diff[:, 2]**2)):.2f} deg")
                    
                    # Angular velocity differences with inversions
                    gyro1 = aligned_data[device1]['gyro_board'].copy()
                    gyro2 = aligned_data[device2]['gyro_board'].copy()
                    
                    if device1_inversions.get('gyro_x', False): gyro1[:, 0] *= -1
                    if device1_inversions.get('gyro_y', False): gyro1[:, 1] *= -1
                    if device1_inversions.get('gyro_z', False): gyro1[:, 2] *= -1
                    
                    if device2_inversions.get('gyro_x', False): gyro2[:, 0] *= -1
                    if device2_inversions.get('gyro_y', False): gyro2[:, 1] *= -1
                    if device2_inversions.get('gyro_z', False): gyro2[:, 2] *= -1
                    
                    gyro_diff = gyro1 - gyro2
                    gyro_diff_deg = gyro_diff * 180/np.pi
                    
                    print("  Angular Velocity RMS Differences:")
                    print(f"    omega_x: {np.sqrt(np.mean(gyro_diff_deg[:, 0]**2)):.2f} deg/s")
                    print(f"    omega_y: {np.sqrt(np.mean(gyro_diff_deg[:, 1]**2)):.2f} deg/s")
                    print(f"    omega_z: {np.sqrt(np.mean(gyro_diff_deg[:, 2]**2)):.2f} deg/s")

# ===== UI Creation =====
print("# 单IMU数据管理与对比分析系统")
print("支持单个IMU数据的逐个上传、管理和多设备对比")

# Create data loader instance
data_loader = SingleIMUDataLoader()

# Create UI components for calibration
device_name_input = widgets.Text(
    value='IMU001',
    placeholder='Enter device name',
    description='Device Name:',
    style={'description_width': 'initial'}
)

sample_rate_widget = widgets.FloatText(
    value=30.0,
    min=1.0,
    max=1000.0,
    description='Sample Rate (Hz):',
    style={'description_width': 'initial'}
)

# Filter controls
filter_enabled_widget = widgets.Checkbox(
    value=True,
    description='Enable Low-pass Filter',
    style={'description_width': 'initial'}
)

filter_cutoff_widget = widgets.FloatSlider(
    value=10.0,
    min=0.1,
    max=50.0,
    step=0.1,
    description='Cutoff Frequency (Hz):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

filter_order_widget = widgets.IntSlider(
    value=4,
    min=1,
    max=8,
    step=1,
    description='Filter Order:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Static data upload
static_upload = widgets.FileUpload(
    accept='.csv,.txt',
    multiple=False,
    description='Select Static Data'
)
static_output = widgets.Output()
static_preview_btn = widgets.Button(
    description="Preview Static Data",
    button_style='primary',
    icon='eye',
    disabled=True
)

# Rotation data upload
rotation_upload = widgets.FileUpload(
    accept='.csv,.txt',
    multiple=False,
    description='Select Rotation Data'
)
rotation_output = widgets.Output()
rotation_preview_btn = widgets.Button(
    description="Preview Rotation Data",
    button_style='primary',
    icon='eye',
    disabled=True
)

# Calibration button
calibrate_btn = widgets.Button(description="Run Calibration", button_style='success')

# Status display
status_output = widgets.Output()

# Results display
results_output = widgets.Output()

# Check calibration status button
check_calibration_btn = widgets.Button(description="Check Calibration Status", button_style='info')

# ===== Event Handlers for Calibration =====
def update_sample_rate(change):
    """Update sample rate"""
    data_loader.sample_rate = change['new']
    filter_cutoff_widget.max = change['new'] / 2 - 0.1
    with status_output:
        clear_output()
        print(f"Sample rate updated to: {change['new']} Hz")
        print(f"Maximum filter cutoff: {filter_cutoff_widget.max:.1f} Hz")

def update_device_name(change):
    """Update device name"""
    data_loader.current_device_name = change['new']
    with status_output:
        clear_output()
        print(f"Device name set to: {change['new']}")

def update_filter_settings(change):
    """Update filter settings"""
    data_loader.filter_enabled = filter_enabled_widget.value
    data_loader.filter_cutoff = filter_cutoff_widget.value
    data_loader.filter_order = filter_order_widget.value
    
    filter_cutoff_widget.disabled = not filter_enabled_widget.value
    filter_order_widget.disabled = not filter_enabled_widget.value
    
    with status_output:
        clear_output()
        if data_loader.filter_enabled:
            print(f"Filter enabled: Butterworth low-pass")
            print(f"Cutoff frequency: {data_loader.filter_cutoff} Hz")
            print(f"Filter order: {data_loader.filter_order}")
        else:
            print("Filter disabled")

def handle_static_upload(change):
    """Handle static data upload"""
    with static_output:
        clear_output()
        if len(static_upload.value) > 0:
            try:
                uploaded_file = static_upload.value[0]
                file_content = uploaded_file['content']
                
                data_loader.current_device_name = device_name_input.value
                result = data_loader.load_data(file_content, 'static')
                
                if result is not None:
                    static_preview_btn.disabled = False
                    with status_output:
                        clear_output()
                        print(f"✅ Static data loaded successfully for {data_loader.current_device_name}!")
                        print(f"Samples: {len(data_loader.static_data)}")
                else:
                    static_preview_btn.disabled = True
                    with status_output:
                        clear_output()
                        print("❌ Failed to load static data")
                    
            except Exception as e:
                static_preview_btn.disabled = True
                with status_output:
                    clear_output()
                    print(f"❌ Error loading static data: {e}")

def handle_rotation_upload(change):
    """Handle rotation data upload"""
    with rotation_output:
        clear_output()
        if len(rotation_upload.value) > 0:
            try:
                uploaded_file = rotation_upload.value[0]
                file_content = uploaded_file['content']
                
                data_loader.current_device_name = device_name_input.value
                result = data_loader.load_data(file_content, 'rotation')
                
                if result is not None:
                    rotation_preview_btn.disabled = False
                    with status_output:
                        clear_output()
                        print(f"✅ Rotation data loaded successfully for {data_loader.current_device_name}!")
                        print(f"Samples: {len(data_loader.rotation_data)}")
                else:
                    rotation_preview_btn.disabled = True
                    with status_output:
                        clear_output()
                        print("❌ Failed to load rotation data")
                    
            except Exception as e:
                rotation_preview_btn.disabled = True
                with status_output:
                    clear_output()
                    print(f"❌ Error loading rotation data: {e}")

def preview_static_data(b):
    """Preview static data"""
    with static_output:
        clear_output()
        if data_loader.static_data is not None:
            data_loader.preview_data('static')
        else:
            print("No static data loaded")

def preview_rotation_data(b):
    """Preview rotation data"""
    with rotation_output:
        clear_output()
        if data_loader.rotation_data is not None:
            data_loader.preview_data('rotation')
        else:
            print("No rotation data loaded")

def run_calibration(b):
    """Run calibration process"""
    global calibration_results
    
    with results_output:
        clear_output()
        
        if data_loader.static_data is None or data_loader.rotation_data is None:
            print("❌ Please load both static and rotation data first")
            return
        
        device_name = device_name_input.value
        if not device_name:
            print("❌ Please enter a device name")
            return
            
        try:
            print(f"Starting calibration process for {device_name}...")
            if data_loader.filter_enabled:
                print(f"Using filtered data (cutoff={data_loader.filter_cutoff}Hz, order={data_loader.filter_order})")
            else:
                print("Using unfiltered data")
            
            # Get data for calibration
            static_accel = data_loader.get_static_accel()
            rotation_gyro = data_loader.get_rotation_gyro()
            
            if static_accel is None or rotation_gyro is None:
                print(f"❌ No data found")
                return
            
            if len(static_accel) == 0 or len(rotation_gyro) == 0:
                print(f"❌ Empty data")
                return
            
            print(f"Static acceleration data shape: {static_accel.shape}")
            print(f"Rotation gyroscope data shape: {rotation_gyro.shape}")
            
            # Run calibration
            R_board_to_imu, angles = calibrate_imu_to_board(static_accel, rotation_gyro)
            
            # Save calibration results
            calibration_results[device_name] = {
                'R_board_to_imu': R_board_to_imu.copy(),
                'R_imu_to_board': R_board_to_imu.T.copy(),
                'angles': angles.copy(),
                'device_name': device_name,
                'filter_settings': {
                    'enabled': data_loader.filter_enabled,
                    'cutoff': data_loader.filter_cutoff,
                    'order': data_loader.filter_order
                }
            }
            
            # Display results
            print("\n=== Calibration Results ===")
            print(f"Device: {device_name}")
            print("Rotation Matrix (Board to IMU):")
            print(np.array2string(R_board_to_imu, precision=4, suppress_small=True))
            
            print("\nInstallation Angles (degrees):")
            print(f"Roll (X): {angles[0]:.2f}°")
            print(f"Pitch (Y): {angles[1]:.2f}°")
            print(f"Yaw (Z): {angles[2]:.2f}°")
            
            # 3D Visualization
            fig = plt.figure(figsize=(10, 8))
            ax = fig.add_subplot(111, projection='3d')
            
            # Define snowboard dimensions
            board_length = 1.6
            board_width = 0.3
            board_thickness = 0.05
            
            # Define IMU dimensions
            imu_size = 0.06
            imu_height = 0.02
            
            # IMU position on board
            imu_pos_board = np.array([0, 0, board_thickness/2 + imu_height/2 + 0.01])
            
            # Create snowboard vertices
            board_vertices = np.array([
                [-board_length/2, -board_width/2, -board_thickness/2],
                [board_length/2, -board_width/2, -board_thickness/2],
                [board_length/2, board_width/2, -board_thickness/2],
                [-board_length/2, board_width/2, -board_thickness/2],
                [-board_length/2, -board_width/2, board_thickness/2],
                [board_length/2, -board_width/2, board_thickness/2],
                [board_length/2, board_width/2, board_thickness/2],
                [-board_length/2, board_width/2, board_thickness/2]
            ])
            
            # Define faces of the board
            board_faces = [
                [board_vertices[0], board_vertices[1], board_vertices[5], board_vertices[4]],
                [board_vertices[1], board_vertices[2], board_vertices[6], board_vertices[5]],
                [board_vertices[2], board_vertices[3], board_vertices[7], board_vertices[6]],
                [board_vertices[3], board_vertices[0], board_vertices[4], board_vertices[7]],
                [board_vertices[4], board_vertices[5], board_vertices[6], board_vertices[7]],
                [board_vertices[3], board_vertices[2], board_vertices[1], board_vertices[0]]
            ]
            
            # Create IMU vertices
            imu_vertices_local = np.array([
                [-imu_size/2, -imu_size/2, -imu_height/2],
                [imu_size/2, -imu_size/2, -imu_height/2],
                [imu_size/2, imu_size/2, -imu_height/2],
                [-imu_size/2, imu_size/2, -imu_height/2],
                [-imu_size/2, -imu_size/2, imu_height/2],
                [imu_size/2, -imu_size/2, imu_height/2],
                [imu_size/2, imu_size/2, imu_height/2],
                [-imu_size/2, imu_size/2, imu_height/2]
            ])
            
            # Transform IMU vertices to board coordinate system
            R_imu_to_board = R_board_to_imu.T
            imu_vertices = np.array([R_imu_to_board @ v + imu_pos_board for v in imu_vertices_local])
            
            # Define faces of the IMU
            imu_faces = [
                [imu_vertices[0], imu_vertices[1], imu_vertices[5], imu_vertices[4]],
                [imu_vertices[1], imu_vertices[2], imu_vertices[6], imu_vertices[5]],
                [imu_vertices[2], imu_vertices[3], imu_vertices[7], imu_vertices[6]],
                [imu_vertices[3], imu_vertices[0], imu_vertices[4], imu_vertices[7]],
                [imu_vertices[4], imu_vertices[5], imu_vertices[6], imu_vertices[7]],
                [imu_vertices[3], imu_vertices[2], imu_vertices[1], imu_vertices[0]]
            ]
            
            # Plot snowboard
            board_collection = Poly3DCollection(board_faces, alpha=0.3, facecolor='#8B4513', edgecolor='#654321')
            ax.add_collection3d(board_collection)
            
            # Plot IMU
            imu_collection = Poly3DCollection(imu_faces, alpha=0.7, facecolor='#808080', edgecolor='#000000')
            ax.add_collection3d(imu_collection)
            
            # Plot coordinate axes
            axis_length = 0.5
            axis_width = 3
            
            # Board coordinate axes
            ax.quiver(0, 0, 0, axis_length, 0, 0, color='red', arrow_length_ratio=0.1, linewidth=axis_width)
            ax.quiver(0, 0, 0, 0, axis_length, 0, color='green', arrow_length_ratio=0.1, linewidth=axis_width)
            ax.quiver(0, 0, 0, 0, 0, axis_length, color='blue', arrow_length_ratio=0.1, linewidth=axis_width)
            
            # IMU coordinate axes
            xi_in_board = R_imu_to_board @ np.array([1, 0, 0])
            yi_in_board = R_imu_to_board @ np.array([0, 1, 0])
            zi_in_board = R_imu_to_board @ np.array([0, 0, 1])
            
            imu_axis_length = 0.3
            ax.quiver(imu_pos_board[0], imu_pos_board[1], imu_pos_board[2], 
                     xi_in_board[0]*imu_axis_length, xi_in_board[1]*imu_axis_length, xi_in_board[2]*imu_axis_length, 
                     color='red', arrow_length_ratio=0.15, linewidth=2, linestyle='--', alpha=0.7)
            ax.quiver(imu_pos_board[0], imu_pos_board[1], imu_pos_board[2], 
                     yi_in_board[0]*imu_axis_length, yi_in_board[1]*imu_axis_length, yi_in_board[2]*imu_axis_length, 
                     color='green', arrow_length_ratio=0.15, linewidth=2, linestyle='--', alpha=0.7)
            ax.quiver(imu_pos_board[0], imu_pos_board[1], imu_pos_board[2], 
                     zi_in_board[0]*imu_axis_length, zi_in_board[1]*imu_axis_length, zi_in_board[2]*imu_axis_length, 
                     color='blue', arrow_length_ratio=0.15, linewidth=2, linestyle='--', alpha=0.7)
            
            # Labels
            ax.text(axis_length*1.1, 0, 0, 'xb', color='red', fontsize=12, weight='bold')
            ax.text(0, axis_length*1.1, 0, 'yb', color='green', fontsize=12, weight='bold')
            ax.text(0, 0, axis_length*1.1, 'zb', color='blue', fontsize=12, weight='bold')
            
            # Set labels and title
            ax.set_xlabel('Board X (m)', fontsize=12)
            ax.set_ylabel('Board Y (m)', fontsize=12)
            ax.set_zlabel('Board Z (m)', fontsize=12)
            ax.set_title(f'IMU Installation on Snowboard - {device_name}', fontsize=14, weight='bold')
            
            # Legend
            board_patch = mpatches.Patch(color='#8B4513', alpha=0.3, label='Snowboard')
            imu_patch = mpatches.Patch(color='#808080', alpha=0.7, label=f'IMU ({device_name})')
            ax.legend(handles=[board_patch, imu_patch], loc='upper right')
            
            # Set view
            ax.set_box_aspect([1, 1, 0.3])
            ax.view_init(elev=25, azim=45)
            
            # Set axis limits
            max_dim = max(board_length, board_width) * 0.7
            ax.set_xlim([-max_dim, max_dim])
            ax.set_ylim([-max_dim, max_dim])
            ax.set_zlim([-0.2, 0.4])
            
            ax.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
            print(f"\n✅ Calibration completed successfully!")
            print(f"✅ Calibration results saved for {device_name}")
            print(f"Total calibrated devices: {len(calibration_results)}")
            print(f"Calibrated devices: {list(calibration_results.keys())}")
            
        except Exception as e:
            print(f"❌ Calibration failed: {e}")
            import traceback
            traceback.print_exc()

def check_calibration_status(b):
    """Check calibration status"""
    with status_output:
        clear_output()
        print("=== Calibration Status ===")
        print(f"Total calibrated devices: {len(calibration_results)}")
        if calibration_results:
            print("\nCalibrated devices:")
            for device, data in calibration_results.items():
                angles = data['angles']
                filter_settings = data.get('filter_settings', {})
                print(f"  {device}:")
                print(f"    Roll: {angles[0]:.2f}°, Pitch: {angles[1]:.2f}°, Yaw: {angles[2]:.2f}°")
                if filter_settings.get('enabled', False):
                    print(f"    Filter: {filter_settings['cutoff']}Hz, order {filter_settings['order']}")
        else:
            print("No devices calibrated yet.")

# Bind event handlers
device_name_input.observe(update_device_name, names='value')
sample_rate_widget.observe(update_sample_rate, names='value')
filter_enabled_widget.observe(update_filter_settings, names='value')
filter_cutoff_widget.observe(update_filter_settings, names='value')
filter_order_widget.observe(update_filter_settings, names='value')
static_upload.observe(handle_static_upload, names='value')
rotation_upload.observe(handle_rotation_upload, names='value')
static_preview_btn.on_click(preview_static_data)
rotation_preview_btn.on_click(preview_rotation_data)
calibrate_btn.on_click(run_calibration)
check_calibration_btn.on_click(check_calibration_status)

# Display calibration UI
print("\n## 校准设置")
display(widgets.HBox([device_name_input, sample_rate_widget]))

print("\n## 滤波设置")
display(filter_enabled_widget)
display(filter_cutoff_widget)
display(filter_order_widget)

print("\n## 1. 上传静态数据")
print("雪板水平放置时的数据")
print("注：静态数据需要包含包含重力的加速度 Ax(m/s²), Ay(m/s²), Az(m/s²)")
display(widgets.VBox([
    static_upload,
    static_preview_btn,
    static_output
]))

print("\n## 2. 上传旋转数据")
print("绕雪板前进方向旋转时的数据")
print("注：旋转数据需要包含姿态相关的加速度 aX(m/s²), aY(m/s²), aZ(m/s²) 和角速度 Gx(°/s), Gy(°/s), Gz(°/s)")
display(widgets.VBox([
    rotation_upload,
    rotation_preview_btn,
    rotation_output
]))

print("\n## 3. 运行校准")
display(widgets.HBox([calibrate_btn, check_calibration_btn]))

print("\n## 状态信息")
display(status_output)

print("\n## 校准结果")
display(results_output)

# 初始检查
check_calibration_status(None)
update_filter_settings(None)



# 单IMU数据管理与对比分析系统
支持单个IMU数据的逐个上传、管理和多设备对比

## 校准设置



## 滤波设置


Checkbox(value=True, description='Enable Low-pass Filter', style=CheckboxStyle(description_width='initial'))

FloatSlider(value=10.0, description='Cutoff Frequency (Hz):', layout=Layout(width='400px'), max=50.0, min=0.1,…

IntSlider(value=4, description='Filter Order:', layout=Layout(width='400px'), max=8, min=1, style=SliderStyle(…

...


## 3. 运行校准



## 状态信息


Output()


## 校准结果


Output()

In [3]:
# ===== Motion Analysis Module =====
print("\n\n# ========== 单IMU运动数据分析模块 ==========")
print("逐个上传IMU运动数据并进行对比分析")

# Create motion analyzer
if 'calibration_results' in globals() and len(calibration_results) > 0:
    motion_analyzer = SingleIMUMotionAnalyzer(calibration_results)
    print(f"Motion analyzer initialized with {len(calibration_results)} calibrated device(s)")
else:
    print("Warning: No calibration results found. Please complete calibration first.")
    motion_analyzer = None

# Motion analysis UI components
motion_device_selector = widgets.Dropdown(
    options=list(calibration_results.keys()) if calibration_results else [],
    description='Select Calibrated Device:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

motion_sample_rate = widgets.FloatText(
    value=30.0,
    min=1.0,
    max=1000.0,
    description='Sample Rate (Hz):',
    style={'description_width': 'initial'}
)

# Motion filter controls
motion_filter_enabled = widgets.Checkbox(
    value=True,
    description='Enable Low-pass Filter',
    style={'description_width': 'initial'}
)

motion_filter_cutoff = widgets.FloatSlider(
    value=10.0,
    min=0.1,
    max=50.0,
    step=0.1,
    description='Cutoff Frequency (Hz):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

motion_filter_order = widgets.IntSlider(
    value=4,
    min=1,
    max=8,
    step=1,
    description='Filter Order:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Motion data upload
motion_upload = widgets.FileUpload(
    accept='.csv,.txt',
    multiple=False,
    description='Select Motion Data'
)

motion_output = widgets.Output()

# Add motion data button
add_motion_btn = widgets.Button(
    description="Add Motion Data",
    button_style='success',
    icon='plus'
)

# IMU data management
manage_output = widgets.Output()

# Device checkboxes for comparison
device_checkboxes = {}

# Angle inversion settings UI
inversion_output = widgets.Output()

# Analysis buttons
compare_btn = widgets.Button(
    description="Compare Selected IMUs",
    button_style='primary',
    icon='bar-chart'
)

configure_inversions_btn = widgets.Button(
    description="Configure Angle Inversions",
    button_style='warning',
    icon='refresh'
)

# Results output
motion_results = widgets.Output()

# ===== Motion Analysis Event Handlers =====
def update_motion_sample_rate(change):
    """Update motion data sample rate"""
    if motion_analyzer:
        motion_analyzer.sample_rate = change['new']
        motion_filter_cutoff.max = change['new'] / 2 - 0.1
    with motion_output:
        clear_output()
        print(f"Sample rate updated to: {change['new']} Hz")
        print(f"Maximum filter cutoff: {motion_filter_cutoff.max:.1f} Hz")

def update_motion_filter(change):
    """Update motion filter settings"""
    if motion_analyzer:
        motion_analyzer.filter_enabled = motion_filter_enabled.value
        motion_analyzer.filter_cutoff = motion_filter_cutoff.value
        motion_analyzer.filter_order = motion_filter_order.value
    
    motion_filter_cutoff.disabled = not motion_filter_enabled.value
    motion_filter_order.disabled = not motion_filter_enabled.value
    
    with motion_output:
        clear_output()
        if motion_analyzer and motion_analyzer.filter_enabled:
            print(f"Filter enabled: Butterworth low-pass")
            print(f"Cutoff frequency: {motion_analyzer.filter_cutoff} Hz")
            print(f"Filter order: {motion_analyzer.filter_order}")
        else:
            print("Filter disabled")

def update_device_options():
    """Update device selector options"""
    if calibration_results:
        motion_device_selector.options = list(calibration_results.keys())
    else:
        motion_device_selector.options = []

def refresh_imu_list():
    """Refresh the list of loaded IMU data"""
    with manage_output:
        clear_output()
        print("=== Loaded IMU Motion Data ===")
        print(f"Total IMUs loaded: {len(single_imu_motion_data)}\n")
        
        if single_imu_motion_data:
            # Create device selection checkboxes
            global device_checkboxes
            device_checkboxes = {}
            
            checkbox_list = []
            for device, data in single_imu_motion_data.items():
                # Create checkbox for each device
                checkbox = widgets.Checkbox(
                    value=True,
                    description=device,
                    style={'description_width': 'initial'},
                    layout=widgets.Layout(width='200px')
                )
                device_checkboxes[device] = checkbox
                
                # Create delete button
                delete_btn = widgets.Button(
                    description=f"Delete",
                    button_style='danger',
                    icon='trash',
                    layout=widgets.Layout(width='80px')
                )
                delete_btn.device_name = device  # Store device name in button
                delete_btn.on_click(delete_imu_data)
                
                # Info text
                info_text = widgets.Label(
                    value=f"{len(data['time'])} samples, {data['time'][-1]:.1f}s, uploaded at {data['upload_time']}",
                    layout=widgets.Layout(width='400px')
                )
                
                # Create horizontal box for each device
                device_box = widgets.HBox([checkbox, delete_btn, info_text])
                checkbox_list.append(device_box)
            
            # Display all devices
            display(widgets.VBox(checkbox_list))
            
            # Select all / Deselect all buttons
            select_all_btn = widgets.Button(description="Select All", button_style='info')
            deselect_all_btn = widgets.Button(description="Deselect All", button_style='warning')
            
            def select_all(b):
                for cb in device_checkboxes.values():
                    cb.value = True
                    
            def deselect_all(b):
                for cb in device_checkboxes.values():
                    cb.value = False
            
            select_all_btn.on_click(select_all)
            deselect_all_btn.on_click(deselect_all)
            
            display(widgets.HBox([select_all_btn, deselect_all_btn]))
        else:
            print("No IMU motion data loaded yet.")

def delete_imu_data(b):
    """Delete IMU motion data"""
    device_name = b.device_name
    if device_name in single_imu_motion_data:
        del single_imu_motion_data[device_name]
        print(f"Deleted motion data for {device_name}")
        refresh_imu_list()

def add_motion_data(b):
    """Add motion data for selected device"""
    with motion_output:
        clear_output()
        if len(motion_upload.value) > 0 and motion_analyzer is not None:
            device_name = motion_device_selector.value
            if not device_name:
                print("❌ Please select a calibrated device first")
                return
                
            try:
                uploaded_file = motion_upload.value[0]
                file_content = uploaded_file['content']
                
                print(f"Processing motion data for {device_name}...")
                success = motion_analyzer.process_single_imu(file_content, device_name)
                
                if success:
                    print(f"\n✅ Motion data added successfully for {device_name}!")
                    refresh_imu_list()
                    # Clear the upload widget
                    motion_upload.value = ()
                else:
                    print(f"❌ Failed to process motion data for {device_name}")
                    
            except Exception as e:
                print(f"❌ Error: {e}")
                import traceback
                traceback.print_exc()
        else:
            if motion_analyzer is None:
                print("❌ Please complete calibration first")
            else:
                print("❌ Please select a file to upload")

def configure_inversions(b):
    """Configure angle inversions for each device"""
    with inversion_output:
        clear_output()
        if motion_analyzer is None:
            print("❌ Please complete calibration first")
            return
            
        print("=== Configure Angle Inversions ===")
        print("Select which angles to invert for each device:\n")
        
        if single_imu_motion_data:
            inversion_widgets = {}
            
            for device in single_imu_motion_data.keys():
                # Create a label for the device
                device_label = widgets.Label(
                    value=f"{device}:",
                    layout=widgets.Layout(width='150px')
                )
                
                # Create checkboxes for each angle
                roll_cb = widgets.Checkbox(
                    value=motion_analyzer.angle_inversions.get(device, {}).get('roll', False),
                    description='Roll',
                    layout=widgets.Layout(width='80px')
                )
                pitch_cb = widgets.Checkbox(
                    value=motion_analyzer.angle_inversions.get(device, {}).get('pitch', False),
                    description='Pitch',
                    layout=widgets.Layout(width='80px')
                )
                yaw_cb = widgets.Checkbox(
                    value=motion_analyzer.angle_inversions.get(device, {}).get('yaw', False),
                    description='Yaw',
                    layout=widgets.Layout(width='80px')
                )
                
                # Gyro inversions
                gyro_x_cb = widgets.Checkbox(
                    value=motion_analyzer.angle_inversions.get(device, {}).get('gyro_x', False),
                    description='Gyro X',
                    layout=widgets.Layout(width='90px')
                )
                gyro_y_cb = widgets.Checkbox(
                    value=motion_analyzer.angle_inversions.get(device, {}).get('gyro_y', False),
                    description='Gyro Y',
                    layout=widgets.Layout(width='90px')
                )
                gyro_z_cb = widgets.Checkbox(
                    value=motion_analyzer.angle_inversions.get(device, {}).get('gyro_z', False),
                    description='Gyro Z',
                    layout=widgets.Layout(width='90px')
                )
                
                # Store widgets
                inversion_widgets[device] = {
                    'roll': roll_cb,
                    'pitch': pitch_cb,
                    'yaw': yaw_cb,
                    'gyro_x': gyro_x_cb,
                    'gyro_y': gyro_y_cb,
                    'gyro_z': gyro_z_cb
                }
                
                # Create horizontal box for this device
                device_box = widgets.HBox([
                    device_label, roll_cb, pitch_cb, yaw_cb,
                    widgets.Label(value='  |  '),
                    gyro_x_cb, gyro_y_cb, gyro_z_cb
                ])
                display(device_box)
            
            # Apply button
            apply_btn = widgets.Button(
                description="Apply Inversions",
                button_style='success',
                icon='check'
            )
            
            def apply_inversions(b):
                # Update motion analyzer with new inversions
                for device, widgets_dict in inversion_widgets.items():
                    motion_analyzer.angle_inversions[device] = {
                        'roll': widgets_dict['roll'].value,
                        'pitch': widgets_dict['pitch'].value,
                        'yaw': widgets_dict['yaw'].value,
                        'gyro_x': widgets_dict['gyro_x'].value,
                        'gyro_y': widgets_dict['gyro_y'].value,
                        'gyro_z': widgets_dict['gyro_z'].value
                    }
                print("\n✅ Angle inversions updated successfully!")
                
                # Show current settings
                print("\nCurrent inversion settings:")
                for device, inversions in motion_analyzer.angle_inversions.items():
                    active_inversions = [k for k, v in inversions.items() if v]
                    if active_inversions:
                        print(f"  {device}: {', '.join(active_inversions)}")
            
            apply_btn.on_click(apply_inversions)
            
            # Clear all button
            clear_all_btn = widgets.Button(
                description="Clear All",
                button_style='danger',
                icon='times'
            )
            
            def clear_all_inversions(b):
                for device, widgets_dict in inversion_widgets.items():
                    for checkbox in widgets_dict.values():
                        checkbox.value = False
                motion_analyzer.angle_inversions.clear()
                print("✅ All inversions cleared!")
            
            clear_all_btn.on_click(clear_all_inversions)
            
            display(widgets.HBox([apply_btn, clear_all_btn]))
        else:
            print("No IMU data loaded yet.")

def compare_imus(b):
    """Compare selected IMUs"""
    with motion_results:
        clear_output()
        if motion_analyzer and device_checkboxes:
            # Get selected devices
            selected_devices = [device for device, cb in device_checkboxes.items() if cb.value]
            
            if len(selected_devices) == 0:
                print("❌ Please select at least one IMU for comparison")
            else:
                print(f"Comparing {len(selected_devices)} IMU(s)...")
                motion_analyzer.plot_multi_imu_comparison(selected_devices)
        else:
            if motion_analyzer is None:
                print("❌ Please complete calibration first")
            else:
                print("❌ No IMU data available for comparison")

# Bind event handlers
motion_sample_rate.observe(update_motion_sample_rate, names='value')
motion_filter_enabled.observe(update_motion_filter, names='value')
motion_filter_cutoff.observe(update_motion_filter, names='value')
motion_filter_order.observe(update_motion_filter, names='value')
add_motion_btn.on_click(add_motion_data)
compare_btn.on_click(compare_imus)
configure_inversions_btn.on_click(configure_inversions)

# Display motion analysis UI
print("\n## 运动数据设置")
display(widgets.HBox([motion_device_selector, motion_sample_rate]))

# Update device options
update_device_options()

print("\n## 运动数据滤波设置")
display(motion_filter_enabled)
display(motion_filter_cutoff)
display(motion_filter_order)

print("\n## 上传运动数据")
print("为已校准的设备逐个上传运动数据")
print("注：运动数据需要包含姿态相关的加速度 aX(m/s²), aY(m/s²), aZ(m/s²)")
display(widgets.VBox([motion_upload, add_motion_btn, motion_output]))

print("\n## IMU数据管理")
print("查看和管理已上传的IMU数据")
display(manage_output)
refresh_imu_list()

print("\n## 对比分析")
print("可以选择反转特定设备的角度（例如：将 IMU004 的 Roll 角反向显示）")
display(widgets.HBox([compare_btn, configure_inversions_btn]))

print("\n## 角度反转设置")
display(inversion_output)

print("\n## 分析结果")
display(motion_results)

# Status check
if 'calibration_results' in globals() and len(calibration_results) > 0:
    print(f"\n✅ 系统就绪")
    print(f"已校准设备: {list(calibration_results.keys())}")
    print(f"已加载运动数据: {list(single_imu_motion_data.keys())}")
    
    # Example: Set specific inversions programmatically
    # To invert IMU004's roll angle as requested:
    if motion_analyzer and 'IMU004' in single_imu_motion_data:
        motion_analyzer.angle_inversions['IMU004'] = {'roll': True, 'pitch': True, 'yaw': False, 
                                                      'gyro_x': True, 'gyro_y': True, 'gyro_z': False}
        print("\n📌 示例：已将 IMU004 的 Roll 角设置为反向显示")
else:
    print("\n⚠️ 请先完成IMU校准")

# Initialize filter settings
update_motion_filter(None)



# ========== 单IMU运动数据分析模块 ==========
逐个上传IMU运动数据并进行对比分析
Motion analyzer initialized with 1 calibrated device(s)

## 运动数据设置



## 运动数据滤波设置


Checkbox(value=True, description='Enable Low-pass Filter', style=CheckboxStyle(description_width='initial'))

FloatSlider(value=10.0, description='Cutoff Frequency (Hz):', layout=Layout(width='400px'), max=50.0, min=0.1,…

IntSlider(value=4, description='Filter Order:', layout=Layout(width='400px'), max=8, min=1, style=SliderStyle(…

...


## 角度反转设置


Output()


## 分析结果


Output()


✅ 系统就绪
已校准设备: ['IMU002']
已加载运动数据: ['IMU002']
